### Startup

In [ ]:
try:
    import breaching
except ModuleNotFoundError:
    # You only really need this safety net if you want to run these notebooks directly in the examples directory
    # Don't worry about this if you installed the package or moved the notebook to the main directory.
    import os; os.chdir("../..")
    import breaching
    
import torch
from tqdm import tqdm
import random, json, copy
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

# Redirects logs directly into the jupyter notebook
import logging, sys
logging.basicConfig(level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)], format='%(message)s')
logger = logging.getLogger()

In [ ]:
sensitivity_path = os.getcwd() + "/lm/sensitivity/"
print(sensitivity_path)

### Initialize cfg object and system setup:

This will load the full configuration object. This includes the configuration for the use case and threat model as `cfg.case` and the hyperparameters and implementation of the attack as `cfg.attack`. All parameters can be modified below, or overriden with `overrides=` as if they were cmd-line arguments.

In [ ]:
torch.backends.mps.is_available()

In [ ]:
import torch.backends

cfg = breaching.get_config(overrides=["case=10_causal_lang_training",  "attack=tag"])
device = torch.device(f'cuda') if torch.cuda.is_available() else torch.device('cpu')
torch.backends.cudnn.benchmark = cfg.case.impl.benchmark
setup = dict(device=device, dtype=getattr(torch, cfg.case.impl.dtype))
setup

### Modify config options here

You can use `.attribute` access to modify any of these configurations for the attack, or the case:

In [ ]:
cfg.case.user.num_data_points = 1 # How many sentences?
cfg.case.user.user_idx = 1 # From which user?
cfg.case.data.shape = [16] # This is the sequence length

cfg.case.model = "transformer3f"

# cfg.attack.optim.max_iterations = 12000 # Increasing the number of iterations can help this attack

### Instantiate all parties

The following lines generate "server, "user" and "attacker" objects and print an overview of their configurations.

In [ ]:
model_path = f"./lm/causal-lm/model-tag-{cfg.case.model}.pt"
user, server, model, loss_fn = breaching.cases.construct_case(cfg.case, setup, model_path=model_path)

attacker = breaching.attacks.prepare_attack(server.model, server.loss, cfg.attack, setup)
breaching.utils.overview(server, user, attacker)

### Simulate an attacked FL protocol

This exchange is a simulation of a single query in a federated learning protocol. The server sends out a `server_payload` and the user computes an update based on their private local data. This user update is `shared_data` and contains, for example, the parameter gradient of the model in the simplest case. `true_user_data` is also returned by `.compute_local_updates`, but of course not forwarded to the server or attacker and only used for (our) analysis.

In [ ]:
server_payload = server.distribute_payload()
shared_data, true_user_data = user.compute_local_updates(server_payload)

In [ ]:
user.print(true_user_data)

In [ ]:
num_layer = len(model.state_dict())
print(f"Number of Layers: {num_layer}")

In [ ]:
num_param = 0
num_param_list = []
for layer in shared_data['gradients']:
    num_param += layer.numel()
    num_param_list.append(layer.numel())
print(f"Number of Parameters: {num_param}")

## DP Module

In [ ]:
from pathlib import Path

def add_laplace_noise(grad_list: list, b: float, iter: int):
    file_name = f"./lm/causal-lm/noised_grad/{cfg.case.model}_dp_{b}_user_{cfg.case.user.user_idx}_iter_{iter}.pt"
    file_path = Path(file_name)
    if file_path.exists():
        print(f"Found existing noised gradient for b = {b} and iter = {iter}.")
        return torch.load(file_name)
    else:
        print(f"Generating new noised gradient for b = {b} and iter = {iter}.")
        loc = torch.as_tensor(0.0, **setup)
        scale = torch.as_tensor(b, **setup)
        noise_generator = torch.distributions.laplace.Laplace(loc=loc, scale=scale)
        
        new_grad_list = [grad + noise_generator.sample(grad.shape) for grad in grad_list]
        torch.save(new_grad_list, file_name)
        return new_grad_list

## Full DP

In [ ]:
for b in [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013]:
    results_no_list = []
    for i in tqdm(range(10), desc="Adding Laplace Noise"):
        print(f"Add Laplace noise with b = {b}")
        shared_data_copy = copy.deepcopy(shared_data)
        shared_data_copy["gradients"] = add_laplace_noise(shared_data_copy["gradients"], b=b, iter=i)

        results_no_dict = {
            "accuracy": [], 
            "sacrebleu": [], 
            "feat_mse": [], 
            "google_bleu": [],
            "rouge1": [],
            "rouge2": [],
            "rougeL": [],
            "token_acc": [],
            "token_avg_accuracy": []
        }
        
        for _ in tqdm(range(10), desc=f"b = {b}"):
            reconstructed_user_data, stats = attacker.reconstruct([server_payload],
                                                                [copy.deepcopy(shared_data_copy)],
                                                                server.secrets, 
                                                                dryrun=cfg.dryrun)

            metrics = breaching.analysis.report(reconstructed_user_data,
                                                true_user_data,
                                                [server_payload], 
                                                server.model,
                                                order_batch=True,
                                                compute_full_iip=False, 
                                                cfg_case=cfg.case,
                                                setup=setup)
            for k in results_no_dict.keys():
                results_no_dict[k].append(metrics[k])
            user.print_and_mark_correct(reconstructed_user_data, true_user_data)
        results_no_list.append(results_no_dict)        
    torch.save(results_no_list, f"./lm/results/tag/dp/dp_{b}_{cfg.case.model}_no_protection.pt")

In [ ]:
def flat_tensor_list(grads):
    flat_grads = torch.tensor([], device=grads[0].device)
    for i in grads:
        flat_grads = torch.concat((flat_grads, i.view(-1)), dim = 0)
    return flat_grads

def encrypt_by_index(grad_list: list, idx_list: float, if_plot=False) -> list:
    """
    @grad_list: gradient list to be encrypted
    @idx_list: list of encrypted parameter indices
    @if_plot: whether to plot the encrypted ratio of each layer
    """
    grad_flat = flat_tensor_list(grad_list)

    mask = torch.zeros(grad_flat.shape).to(device=grad_flat.device) == 0
    mask[idx_list] = False

    # Apply the encryption mask
    grad_flat = grad_flat * mask + torch.rand(grad_flat.shape).to(device=grad_flat.device) * ~mask

    # Reshape back
    new_grad_list = []
    start_idx = 0
    ratio_enc_per_layer = []
    for layer_grad in tqdm(grad_list):
        layer_param_num = layer_grad.numel()
        grad_slice = grad_flat[start_idx: start_idx + layer_param_num]
        new_grad_list.append(grad_slice.reshape(layer_grad.shape))

        # Record the number of encrypted parameters of each layer
        mask_slice = mask[start_idx: start_idx + layer_param_num]
        ratio_param_enc_layer = (layer_param_num - mask_slice.sum()) / layer_param_num
        ratio_enc_per_layer.append(ratio_param_enc_layer.item())

        start_idx += layer_param_num

    if if_plot:
        plt.bar(np.arange(1, len(ratio_enc_per_layer)+1), ratio_enc_per_layer)
        plt.xlabel("Layer Index")
        plt.ylabel("Encryption Ratio")
        plt.show()
    return new_grad_list

## Selective Encryption + DP

In [ ]:
def encrypt_selective(sens_list, ratio_enc, b, iter, num_repeat=1):
    num_enc = (int) (ratio_enc * num_param)
    encrypt_list = torch.topk(flat_tensor_list(sens_list).abs(), num_enc, largest=True).indices

    new_gradient = copy.deepcopy(shared_data['gradients'])
    new_gradient = add_laplace_noise(new_gradient, b, iter)
    new_gradient = encrypt_by_index(new_gradient, encrypt_list)
    
    gradient_with_he = list(new_gradient)
    new_shared_data = dict()
    for k in shared_data.keys():
        if k == "gradients":
            new_shared_data[k] = gradient_with_he
        else:
            new_shared_data[k] = shared_data[k]

    results_dict = {
        "accuracy": [], 
        "sacrebleu": [], 
        "feat_mse": [], 
        "google_bleu": [],
        "rouge1": [],
        "rouge2": [],
        "rougeL": [],
        "token_acc": [],
        "token_avg_accuracy": []
    }
    for _ in tqdm(range(num_repeat), desc=f"p={ratio_enc}, b={b}"):
        new_reconstructed_user_data, stats = attacker.reconstruct([server_payload],
                                                                [copy.deepcopy(new_shared_data)],
                                                                server.secrets,
                                                                dryrun=cfg.dryrun)
        print(f"Encrypted the {ratio_enc * 100:.2f}% parameters")
        metrics = breaching.analysis.report(new_reconstructed_user_data,
                                            true_user_data,
                                            [server_payload],
                                            server.model,
                                            order_batch=True,
                                            compute_full_iip=False,
                                            cfg_case=cfg.case,
                                            setup=setup)
        for k in results_dict.keys():
            results_dict[k].append(metrics[k])
        user.print_and_mark_correct(new_reconstructed_user_data, true_user_data)
    return results_dict
        

In [ ]:
sens_path = sensitivity_path + f"{cfg.case.model}_tag_mean_sens.pt"
sens_list = torch.load(sens_path)

for b in [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013]:
    ratio = 0.0001
    results_select_list = []
    for i in tqdm(range(10), desc="Adding Laplace Noise"):
        results_select_dict = encrypt_selective(sens_list, 
                                                ratio_enc=ratio,
                                                b=b,
                                                iter=i,
                                                num_repeat=10)
        results_select_list.append(results_select_dict)
    torch.save(results_select_list, f"./lm/results/tag/dp/dp_{b}_{cfg.case.model}_selective_{ratio}.pt")

## Random Encryption + DP

In [ ]:
# below are for for random encryption by parameters (modified):

def encrypt_random(ratio_enc, b, iter, num_repeat=1):
    # Randomly pick a proportion to encrypt
    protected_params = random.sample(range(num_param), (int) (ratio_enc * num_param))

    new_gradient = copy.deepcopy(shared_data['gradients'])
    new_gradient = add_laplace_noise(new_gradient, b, iter)
    new_gradient = encrypt_by_index(new_gradient, protected_params)

    gradient_with_he = list(new_gradient)
    new_shared_data = dict()
    for k in shared_data.keys():
        if k == "gradients":
            new_shared_data[k] = gradient_with_he
        else:
            new_shared_data[k] = shared_data[k]

    results_dict = {
        "accuracy": [], 
        "sacrebleu": [], 
        "feat_mse": [], 
        "google_bleu": [],
        "rouge1": [],
        "rouge2": [],
        "rougeL": [],
        "token_acc": [],
        "token_avg_accuracy": []
    }

    for _ in tqdm(range(num_repeat), desc=f"p={ratio_enc}, b={b}"):
        new_reconstructed_user_data, stats = attacker.reconstruct([server_payload],
                                                                [copy.deepcopy(new_shared_data)],
                                                                server.secrets,
                                                                dryrun=cfg.dryrun)
        print(f"Randomly encrypted {ratio_enc * 100}% parameters")
        metrics = breaching.analysis.report(new_reconstructed_user_data,
                                            true_user_data,
                                            [server_payload],
                                            server.model,
                                            order_batch=True,
                                            compute_full_iip=False,
                                            cfg_case=cfg.case,
                                            setup=setup)
        
        for k in results_dict.keys():
            results_dict[k].append(metrics[k])
        user.print_and_mark_correct(new_reconstructed_user_data, true_user_data)
    return results_dict

In [ ]:
for b in [0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009, 0.01, 0.011, 0.012, 0.013]:
    ratio = 0.0001
    results_random_list = []
    for i in tqdm(range(10), desc="Adding Laplace Noise"):
        results_random_list_b = []
        for _ in tqdm(range(10), desc="Testing random init..."):
            results_dict = encrypt_random(ratio_enc=ratio, b=b, iter=i, num_repeat=10)
            results_random_list_b.append(results_dict)
        results_random_list.append(results_random_list_b)
    torch.save(results_random_list, f"./lm/results/tag/dp/dp_{b}_{cfg.case.model}_random_{ratio}.pt")